In [24]:
import wrds
import pandas as pd



db = wrds.Connection(wrds_username='jb101')

Loading library list...
Done


In [ ]:

query = """

WITH base AS (
  SELECT *
  FROM trace.trace_enhanced
  WHERE trd_exctn_dt BETWEEN DATE '2015-01-01' AND DATE '2016-12-31'
    AND rptd_pr IS NOT NULL
    AND entrd_vol_qt > 0
),

-- Identify all correction/cancel/reversal/error reports
error_reports AS (
  SELECT *
  FROM base
  WHERE trc_st IN ('C','W','R','X','Y')
),

-- Identify the original "T" trades that match these error reports by 7-key linkage
originals_to_drop AS (
  SELECT DISTINCT t.*
  FROM base t
  JOIN error_reports e
    ON t.cusip_id     = e.cusip_id
   AND t.trd_exctn_dt = e.trd_exctn_dt
   AND t.trd_exctn_tm = e.trd_exctn_tm
   AND t.rptd_pr      = e.rptd_pr
   AND t.entrd_vol_qt = e.entrd_vol_qt
   AND t.rpt_side_cd  = e.rpt_side_cd
   AND t.cntra_mp_id   = e.cntra_mp_id
   AND t.msg_seq_nb   = e.msg_seq_nb
  WHERE t.trc_st = 'T'
),

-- Union the two lists to drop (error reports + their originals)
to_remove AS (
  SELECT cusip_id, trd_exctn_dt, trd_exctn_tm,
         rptd_pr, entrd_vol_qt, rpt_side_cd, cntra_mp_id, msg_seq_nb
  FROM error_reports
  UNION
  SELECT cusip_id, trd_exctn_dt, trd_exctn_tm,
         rptd_pr, entrd_vol_qt, rpt_side_cd, cntra_mp_id, msg_seq_nb
  FROM originals_to_drop
),

-- Keep only clean "T" trades
clean AS (
  SELECT b.*
  FROM base b
  LEFT JOIN to_remove r
    ON b.cusip_id     = r.cusip_id
   AND b.trd_exctn_dt = r.trd_exctn_dt
   AND b.trd_exctn_tm = r.trd_exctn_tm
   AND b.rptd_pr      = r.rptd_pr
   AND b.entrd_vol_qt = r.entrd_vol_qt
   AND b.rpt_side_cd  = r.rpt_side_cd
   AND b.cntra_mp_id   = r.cntra_mp_id
   AND b.msg_seq_nb   = r.msg_seq_nb
  WHERE r.cusip_id IS NULL
),

-- Filter hours, price, etc.
step_hours AS (
  SELECT *
  FROM clean
  WHERE CAST(trd_exctn_tm AS TIME) BETWEEN TIME '08:00:00' AND TIME '17:15:00'
),
step_price AS (
  SELECT *
  FROM step_hours
  WHERE rptd_pr >= 10
),
step_subprd AS (
  SELECT *
  FROM step_price
  WHERE sub_prdct = 'CORP'
),
step_sale AS (
  SELECT *
  FROM step_subprd
  WHERE COALESCE(sale_cndtn_cd,'') NOT IN ('W','L','T','S','P')
),

step_capacity AS (
  SELECT *
  FROM step_sale
  WHERE NOT (
    (buy_cpcty_cd IS DISTINCT FROM sell_cpcty_cd)
    AND buy_cpcty_cd IS NOT NULL
    AND sell_cpcty_cd IS NOT NULL
  )
),

step_agency AS (
  SELECT *
  FROM step_capacity
  WHERE NOT (
        (rpt_side_cd='B' AND buy_cpcty_cd='A')
      )
),

-- Holiday filter using CRSP trading dates
valid_days AS (
  SELECT DISTINCT "date"::date AS trd_exctn_dt
  FROM crsp.dsi
  WHERE "date" BETWEEN DATE '2015-01-01' AND DATE '2016-12-31'
),

step_biz AS (
  SELECT s.*
  FROM step_agency AS s
  JOIN valid_days AS v
    ON s.trd_exctn_dt::date = v.trd_exctn_dt
  WHERE EXTRACT(ISODOW FROM s.trd_exctn_dt) < 6
)

SELECT
  s.cusip_id,
  s.trd_exctn_dt,
  s.trd_exctn_tm,
  s.msg_seq_nb,
  s.rptd_pr,
  s.entrd_vol_qt,
  s.rpt_side_cd,
  COALESCE(s.buy_cpcty_cd, s.sell_cpcty_cd) AS capacity,
  s.trc_st,
  s.sale_cndtn_cd,
  s.sub_prdct
FROM step_biz AS s
ORDER BY s.cusip_id, s.trd_exctn_dt, s.trd_exctn_tm, s.msg_seq_nb
LIMIT 10000;



"""


df = db.raw_sql(query)


df.head()




,cusip_id,trd_exctn_dt,trd_exctn_tm,msg_seq_nb,rptd_pr,entrd_vol_qt,rpt_side_cd,capacity,trc_st,sale_cndtn_cd,sub_prdct
0,000361AM7,2015-01-05,15:06:32,0052184,102.0,100000.0,B,P,T,<NA>,CORP
1,000361AM7,2015-01-13,10:17:36,0012062,100.5,17000.0,B,P,T,<NA>,CORP
2,000361AM7,2015-01-14,15:00:49,0052138,100.5,280000.0,B,P,T,<NA>,CORP
3,000361AM7,2015-01-29,11:34:01,0023933,102.75,15000.0,B,P,T,<NA>,CORP
4,000361AM7,2015-02-03,12:12:23,0035705,104.5448,109000.0,S,P,T,<NA>,CORP


In [ ]:
# Basic cleaning



for i in df.columns:

    if df[i].isna().mean() > 0.95:
        df = df.drop(columns=[i])

    elif df[i].nunique() == 1:
        df = df.drop(columns=[i])

# Could be changed?
df['rpt_side_cd'].map({'B': 1, 'S': -1})





df.head()

,cusip_id,trd_exctn_dt,trd_exctn_tm,msg_seq_nb,rptd_pr,entrd_vol_qt,rpt_side_cd,capacity
0,000361AM7,2015-01-05,15:06:32,0052184,102.0,100000.0,B,P
1,000361AM7,2015-01-13,10:17:36,0012062,100.5,17000.0,B,P
2,000361AM7,2015-01-14,15:00:49,0052138,100.5,280000.0,B,P
3,000361AM7,2015-01-29,11:34:01,0023933,102.75,15000.0,B,P
4,000361AM7,2015-02-03,12:12:23,0035705,104.5448,109000.0,S,P


In [36]:



df['capacity'].value_counts()





capacity
P    9405
A     595
Name: count, dtype: Int64